# Lab 02 · HTTP Forms and Retry

*This lab notebook provides guided steps. All commands are intended for local execution.*

## Objectives
- A robust fetch helper with retry logic is introduced.
- A controlled React form is configured with loading and error feedback.
- Friendly error messages are surfaced in the UI.

## What will be learned
- Retry helpers are structured for frontend HTTP calls.
- Controlled form patterns in React are rehearsed.
- Error boundaries in simple forms are practiced.

## Prerequisites & install
The following commands are intended for local execution.

```bash
cd ai-web/frontend
npm install
```


### FastAPI service context

#### Project modules at a glance
- `app/main.py` instantiates `FastAPI()`, attaches middleware, and mounts routers that expose `/echo`, `/flaky-echo`, and AI-specific routes. [Docs: First Steps](https://fastapi.tiangolo.com/tutorial/first-steps/)
- `app/routers/*.py` files group related path operations (echo, profiles, inference) with `APIRouter`, keeping HTTP contracts colocated with their dependencies. [Docs: Bigger Applications](https://fastapi.tiangolo.com/tutorial/bigger-applications/)
- `app/schemas.py` (or per-router schema modules) defines request and response models with Pydantic so payloads are validated before network calls. [Docs: Request Body](https://fastapi.tiangolo.com/tutorial/body/)
- `app/dependencies.py` centralizes shared resources: AI clients, caches, rate limiters, and database sessions that are injected into routes. [Docs: Dependencies](https://fastapi.tiangolo.com/tutorial/dependencies/)

#### Dependency injection, validation, and error handling
1. **Inject shared services** with `Depends` to supply configured AI providers or retry-aware HTTP clients. This keeps path operations small and testable. [Docs: Dependencies in path operation decorators](https://fastapi.tiangolo.com/tutorial/dependencies/dependencies-with-yield/)
2. **Guard requests** by layering validation logic in Pydantic models (`Field` constraints, enums for model selection) so malformed prompts never reach third-party APIs. [Docs: Extra Data Types](https://fastapi.tiangolo.com/tutorial/extra-data-types/)
3. **Translate failures** into `HTTPException` responses that surface actionable status codes and messages to the React UI. Use exception handlers to map provider timeouts into `503` responses that the retry helper can recognize. [Docs: Handling Errors](https://fastapi.tiangolo.com/tutorial/handling-errors/)
4. **Instrument long tasks** with `BackgroundTasks` for actions like saving transcripts or emitting telemetry once inference completes. [Docs: Background Tasks](https://fastapi.tiangolo.com/tutorial/background-tasks/)
5. **Document everything** via the automatic OpenAPI schema so teammates can inspect request/response shapes directly in `/docs`. [Docs: Interactive API docs](https://fastapi.tiangolo.com/features/#interactive-api-docs)

#### How this supports AI endpoints
- Async path operations let you await inference, embedding, or vector search calls without blocking other requests.
- Dependency-injected providers make it easy to swap models (e.g., OpenAI, Anthropic, local models) or wrap them with retry/backoff logic.
- Consistent error payloads (status code + message + optional retry-after metadata) give the frontend enough context to decide when to retry, alert, or escalate.

### Vite-powered React environment refresher

#### Dev tooling recap
- Initialize the project with `npm create vite@latest frontend -- --template react` and install dependencies (`npm install`). [Docs: Getting Started](https://vitejs.dev/guide/)
- Run `npm run dev` for hot module replacement while coding forms; run `npm run build && npm run preview` to validate production bundles.
- Configure the dev server proxy in `vite.config.js` so `/api` points to `http://localhost:8000`, matching the FastAPI service. [Docs: Server Proxy](https://vitejs.dev/config/server-options.html#server-proxy)
- Store secrets and backend URLs in `.env.local` using `VITE_` prefixes (e.g., `VITE_API_BASE_URL`). Access them in code via `import.meta.env`. [Docs: Env Variables](https://vitejs.dev/guide/env-and-mode.html)

#### Organizing AI-friendly modules
- Mirror backend routers by placing React features in `src/features/<domain>/` with colocated hooks (`useEchoForm.js`, `useProfileFetcher.js`).
- Keep API helpers inside `src/lib/` such as `retry.js`, `apiClient.js`, and streaming utilities that wrap `fetch` with AbortController support.
- Use React hooks (`useState`, `useEffect`, `useReducer`) to manage prompt inputs, loading flags, and streaming buffers. Reference the [React docs](https://react.dev/reference/react) for deeper hook usage.
- Compose UI primitives in `src/components/` for buttons, toasts, and skeleton loaders so each AI workflow shares consistent feedback.
- Track experiment flags (e.g., "use streaming", "call fallback model") with context providers or state libraries (Zustand, Redux Toolkit) when the app grows.

#### Backend ↔ frontend toolchain overview
| Layer | Key tool | Purpose for this lab |
| --- | --- | --- |
| Backend runtime | FastAPI + Uvicorn | Serves async echo routes, enforces validation, exposes OpenAPI docs |
| Shared services | Dependency injection | Provides AI clients, caches, and retry-enabled HTTP wrappers |
| Frontend dev server | Vite HMR proxy | Mirrors backend routes locally, handles env injection |
| UI layer | React components + hooks | Collects form data, surfaces retries, renders AI responses |

### AI request flow checklist
1. **Define the FastAPI route**: confirm `app/routers/echo.py` (or similar) receives a validated model and raises `HTTPException` with descriptive messages for retryable failures.
2. **Register dependencies and middleware**: ensure CORS, logging, and provider clients are available so retries are meaningful and observable.
3. **Expose configuration**: add `.env` values for `API_BASE_URL`, AI model keys, and retry timing that both FastAPI and Vite can read.
4. **Create or update the shared API utility**: in `src/lib/apiClient.js`, read `import.meta.env.VITE_API_BASE_URL`, attach headers, and export helpers consumed by `withRetry`.
5. **Wire React forms to the helper**: in `src/App.jsx`, collect input, call `withRetry`, and map resolved data into component state while disabling submit buttons during attempts.
6. **Iterate locally, then containerize**: run `uvicorn app.main:app --reload` alongside `npm run dev` to test the full round trip. Once stable, bake the same env vars into Docker or deployment pipelines for parity.


## Step-by-step tasks
### Step 1: Retry helper placement
A retry helper is positioned under src/lib.

In [ ]:
from pathlib import Path
lib = Path("ai-web/frontend/src/lib")
lib.mkdir(parents=True, exist_ok=True)
(lib / "retry.js").write_text('''export async function withRetry(fn, attempts = 2, delayMs = 400) {
  let lastError;
  for (let attempt = 0; attempt <= attempts; attempt += 1) {
    try {
      return await fn();
    } catch (error) {
      lastError = error;
    }
    await new Promise((resolve) => setTimeout(resolve, delayMs));
  }
  throw lastError;
}
''')
print("Retry helper was written.")


### Step 2: Form integration
The React form now leans on a reusable `withRetry` helper so network hiccups do not derail the student experience. The flow looks like this:

1. Collect user input in local component state.
2. Call `withRetry` with an async function that posts the form data.
3. Retry the call if a transient failure occurs, backing off briefly between attempts.
4. Show a friendly message if every attempt fails, otherwise render the echoed response.

```jsx
// Flow: collect input -> call withRetry -> update component state with the response
const sendEcho = () =>
  withRetry(
    () => post('/echo', { msg }), // Async callback posting the current message text to FastAPI
    2, // Retry up to two extra attempts for transient 5xx or network errors
    500, // Wait 500 ms between attempts before trying again
  );
```

Because `withRetry` only depends on an async callback, you can scale the pattern across modules. For example, a profile page could share the helper from `src/lib/retry.js` and dial in its own policy:

```jsx
// Flow: fetch profile data with the shared retry helper so the page stays resilient
const fetchProfile = () =>
  withRetry(
    () => api.getProfile(userId), // Invoke a module-level API client using the current userId
    3, // Allow three total attempts to tolerate slow or flaky upstream services
    800, // Back off 800 ms between retries (tweak or expand to exponential backoff as needed)
  );
```

> Tip: pass higher attempt counts and delays when hitting slower services or when layering exponential backoff (`delayMs * attempt`).

Larger forms follow the same controlled-input structure: hold each field in state, validate before submitting, and surface field-level errors next to the relevant inputs. When multiple fields need to submit together, build one payload object and reuse `withRetry` so the entire submission benefits from resilience.

To see retries in action, the backend now exposes `/flaky-echo`. Point the helper at it during testing:

```jsx
// Flow: exercise the flaky endpoint to watch retries resolve the request in development
await withRetry(
  () => post('/flaky-echo?failures=2', { msg }), // Endpoint intentionally fails twice before succeeding
  3, // Provide three total attempts so the third call can succeed
  500, // Pause half a second between attempts to avoid hammering the server
);
```

The first two requests return HTTP 503 errors, the third succeeds, and the UI recovers gracefully. Wrap longer-lived retry sequences in loading spinners, keep buttons disabled until the promise settles, and consider error boundaries to catch truly fatal issues. These guardrails help the codebase grow while staying aligned with the lab’s resilient frontend architecture.

These commented snippets mirror the updates you will apply to `src/App.jsx` (and related hooks) so learners can translate the notebook guidance directly into the working project files.


In [ ]:
from pathlib import Path
app_js = Path("ai-web/frontend/src/App.jsx")
text = app_js.read_text()
if "withRetry" not in text:
    text = text.replace(
        "import { post } from './lib/api';",
        "import { post } from './lib/api';
import { withRetry } from './lib/retry';",
    )
if "withRetry" in text and "withRetry(() => post" not in text:
    text = text.replace(
        "const json = await post('/echo', { msg });",
        "const json = await withRetry(() => post('/echo', { msg }), 2, 500);",
    )
if "setError(String(err));" in text:
    text = text.replace(
        "setError(String(err));",
        "setError('A temporary issue was encountered. Please try again.');",
    )
app_js.write_text(text)
print("App.jsx was adjusted for retry and friendly errors.")

## Validation / acceptance checks
```bash
# locally
curl -X POST http://localhost:8000/echo -H 'Content-Type: application/json' -d '{"msg":"retry"}'
```
- The echoed payload is returned successfully after transient failures are simulated.
- React development mode shows the described UI state without console errors.

## Homework / extensions
- Additional retry backoff strategies are outlined for future reference.
- Form validation rules are drafted to prevent empty submissions.